In [1]:
# ---------------------------------- Install Packages
! pip install pandas
! pip install openpyxl
! pip install xgboost
! pip install scikit-learn
! pip install iteration_utilities
! pip install ta
! pip install MetaTrader5
! pip install openpyxl
! pip install seaborn
! pip install plotly
! pip install ffn
! pip install quantstats
! pip install scikit-optimize
! pip install keras
! pip install pandas_ta
! pip install scikeras

In [2]:
# ---------------------------------- PACKAGES
import MetaTrader5 as mt5
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dropout
# from keras.layers import Dense
from sklearn.feature_selection import RFECV
import ta
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import norm
from sklearn import neighbors
import sys
from pprint import pprint
from sklearn.ensemble import GradientBoostingRegressor
from IPython import display
from skopt import BayesSearchCV
import pandas as pd
import plotly as pl
from datetime import datetime
import plotly.express as px
import plotly
import pandas as pd
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import RandomizedSearchCV
from statistics import stdev
from statistics import variance
import os
import glob
import numpy as np
import warnings
import plotly.graph_objs as go
import quantstats as qs
import plotly.subplots
import seaborn as sns
from bs4 import BeautifulSoup as BS
import urllib.request
import re
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from iteration_utilities import deepflatten
import pandas_ta as ta
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
# from scikeras.wrappers import KerasRegressor
# import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import GRU, Dense
# from keras.layers import LSTM, Dense
# import tensorflow
# import tensorflow.keras.wrappers
# from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.layers import Dropout
import pandas as pd
from skopt.space import Real, Integer, Categorical
from scipy.stats import loguniform
from scipy.stats import uniform
from scipy.stats import randint
from datetime import date

warnings.filterwarnings("ignore")

In [13]:
# ---------------------------------- Data Import
AUDUSD=pd.read_pickle('Data/AUDUSD.pkl')
EURUSD=pd.read_pickle('Data/EURUSD.pkl')
GBPUSD=pd.read_pickle('Data/GBPUSD.pkl')
NZDUSD=pd.read_pickle('Data/NZDUSD.pkl')
USDCAD=pd.read_pickle('Data/USDCAD.pkl')
USDCHF=pd.read_pickle('Data/USDCHF.pkl')


In [15]:
# all_close=pd.concat([AUDUSD['time'],AUDUSD['return'],EURUSD['return'],GBPUSD['return'],NZDUSD['return'], USDCAD['return'], USDCHF['return']],axis=1)
# all_close.columns=['time','AUDUSD','EURUSD','GBPUSD','NZDUSD','USDCAD','USDCHF']
# all_close.to_csv('all_return.csv')


In [14]:
# ---------------------------------- RETURN
for a in [EURUSD, AUDUSD, GBPUSD, NZDUSD,USDCAD,USDCHF]:
    a['close_pre']=a.close.shift(1)
    a['return']=(a['close']-a['close_pre'])/a['close_pre'] # simple return

In [5]:
# --------------------------------- TA
def cci(pair,cci_sma,cci_num):
    cci_num=0.015
    pair['typ.price']=(pair['high']+pair['low']+pair['close'])/3
    pair['cci']=(pair['typ.price']-pair['typ.price'].rolling(cci_sma).mean())/(cci_num*stdev(pair['typ.price']))

def atr(pair,period):
    pair['atr']=ta.atr(pair['high'],pair['low'],pair['close'],period)
# def ichimoku(pair):
#     pair[['ISA1','ISB1','ITS1','IKS1','ICS1']]=ta.ichimoku(pair['high'],pair['low'],pair['close'],tenkan=True,kijun=True,senkou=True)
def willr(pair,period):
    pair['willr']=ta.willr(pair['high'],pair['low'],pair['close'],period)
def adx(pair,period):
    pair[['ADX_14','DMP_14','DMN_14']]=ta.adx(pair['high'],pair['low'],pair['close'],period)
def macd(pair,slow,fast,smooth): # 26,12,9
    pair['macd_slow']=pair['close'].ewm(span=fast).mean()-pair['close'].ewm(span=slow).mean()
    pair['macd_fast']=pair['close'].ewm(span=smooth).mean()
def Stochastic(pair,k_period,d_period):
    pair['sto_dif1']=pair['close']-pair['low'].rolling(k_period).min()
    pair['sto_dif2']=pair['high'].rolling(k_period).max()-pair['low'].rolling(k_period).min()
    pair['sto_k']=pair['sto_dif1']/pair['sto_dif2']*100
    pair['sto_main']=pair['sto_dif1'].rolling(d_period).sum()/pair['sto_dif2'].rolling(d_period).sum()*100
    pair['sto_signal']=pair['sto_main'].rolling(d_period).mean()
def MA(pair,period,type_ma):
    if type_ma=="sma":
        pair['sma'+str(period)]=pair['close'].rolling(period).mean()
    elif type_ma=="ema":
        pair['ema'+str(period)]=pair['close'].ewm(span=period).mean()
def RSI(pair, periods):
    pair['rsi_up']=pair['close']-pair['close'].shift(1) #shift(1)-previous, shift(-1)-next
    pair['rsi_down']=pair['close'].shift(1)-pair['close']
    pair.loc[pair['rsi_up'] <= 0, 'rsi_up'] = 0
    pair.loc[pair['rsi_down'] <= 0, 'rsi_down'] = 0
    pair['rsi_up_avg']=pair['rsi_up'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    pair['rsi_down_avg']=pair['rsi_down'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    for i in range(periods+1,len(pair)):
        pair['rsi_up_avg'][i]=(pair['rsi_up_avg'][i-1]*(periods-1)+pair['rsi_up'][i])/periods
        pair['rsi_down_avg'][i]=(pair['rsi_down_avg'][i-1]*(periods-1)+pair['rsi_down'][i])/periods
    pair['rsi_rs']=pair['rsi_up_avg']/pair["rsi_down_avg"]
    pair['rsi']=100-(100/(1+pair['rsi_rs']))
def BB(pair,avgday,stdmul):
    pair['bb_std']=pair['close'].rolling(avgday).std()
    pair['bb_mid']=pair['close'].rolling(avgday).mean()
    pair['bb_down']=pair['bb_mid']-pair['bb_std']*stdmul
    pair['bb_up']=pair['bb_mid']+pair['bb_std']*stdmul

def ASD(return_in): # Annual standart deviation
  var_output=return_in.var()
  var_output=var_output*252**(1/2) # 4hour - 1512
  return var_output

def calculate_max_drawdown(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns / rolling_max) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

def calculate_sharpe_ratio(return_series,rf): # Sharpe ratio #255, #0.01
    x=return_series-rf
    mean=x.mean()
    sigma = return_series.std()
    return mean / sigma


def calculate_roi(equity): # Return on Investment (ROI)
  x=(equity.iloc[-1]-equity.iloc[0])/equity.iloc[0]
  return x

for a in [EURUSD, AUDUSD, GBPUSD, NZDUSD,USDCAD,USDCHF]:
    MA(a,10,'ema')
    MA(a,20,'ema')
    MA(a,50,"ema")
    MA(a,200,"ema")
    MA(a,100,"ema")
    macd(a,26,12,9)
    RSI(a, 14)
    Stochastic(a,14,3)
    cci(a,20,0.015)
    BB(a,20,2)
    atr(a,14)
    willr(a,14)
    adx(a,14)
    a=a.drop(columns=['sto_dif1', 'sto_dif2','tick_volume', 'spread',
                      'real_volume','rsi_up', 'rsi_down', 'rsi_up_avg', 'rsi_down_avg',
                      'rsi_rs','sto_k','bb_std','typ.price'], axis=1, inplace=True)

In [11]:
# ---------------------------------- List of Model
def all_model(model_name):
     if model_name=='RIDGE':
        model = Ridge()
        param_grid = {'alpha': uniform(0.00001, 1.0)} # https://arxiv.org/pdf/2108.11872.pdf
     elif model_name=='SVM':
        model = SVR()
        param_grid = {'kernel': ['linear', 'rbf','poly'],
                      'C': loguniform(0.1, 10), #file:///C:/Users/User/Downloads/s13321-015-0088-0.pdf
                      'gamma': uniform(0.1, 10), #https://arxiv.org/pdf/2309.06604.pdf
                      'degree':randint(1,4)}
     elif model_name=='KNN':
        model = neighbors.KNeighborsRegressor()
        param_grid = {
            'n_neighbors': randint(1,20),
            'p': [1,3]
            }
     elif model_name=='RF':
        model = RandomForestRegressor()
        param_grid = {'n_estimators':randint(20, 200),
                      'max_features': randint(5,30),
                      'max_depth':randint(1,6),
                      'min_samples_split':randint(2,30)}
     elif model_name=='XGBOOST':
        model=xgb.XGBRegressor()
        param_grid = {'n_estimators':randint(20, 200),
                      'learning_rate':loguniform(0.001,0.5),
                      'max_depth':randint(8,15),
                      'gamma':uniform(0.001,0.02),
                      }
     return model, param_grid

In [20]:
%%time
# ---------------------------------- IS-OOS
from dateutil.relativedelta import relativedelta

#---------- INPUTS
i=USDCHF
daily_range=600 # 200 in 1 year trainset (3 year: D-600, 4H-3600)
#----------

start_train=datetime(2000,1,1)
end_train=datetime(2002,12,31)
start_test=datetime(2003,1,1)
end_test=datetime(2003,12,31)
is_oos=[[start_train,end_train,start_test,end_test]] # start date
is_oos

for j in range(2000):
    start_train=is_oos[j][0]+relativedelta(months=+6)
    end_train=is_oos[j][1]+relativedelta(months=+6)
    end_train=end_train+ relativedelta(day=31)
    start_test=is_oos[j][2]+relativedelta(months=+6)
    end_test=is_oos[j][3]+relativedelta(months=+6)
    end_test=end_test+ relativedelta(day=31)
    if end_test>datetime(2022,12,31):
        break
    is_oos.append([start_train,end_train,start_test,end_test])

is_oos_index=[]
for z in is_oos:
    train=i[(i.time>=z[0]) & (i.time<=z[1])].index.values.tolist()
    test=i[(i.time>=z[2]) & (i.time<=z[3])].index.values.tolist()
    is_oos_index.append([train, test])

print('train+validation: ',is_oos_index[0][0][0],is_oos_index[0][0][-1])
print('test:',is_oos_index[0][1][0],is_oos_index[0][1][-1])

# ---------------------------------- Classic ML models
OUTPUTS=[]
HYPERS=[]
df_summary_OUTPUTS=[]
for m in ['RIDGE', 'SVM','KNN','RF','XGBOOST']: # ,'GBDT'
    model, param_grid = all_model(m)
    i=i[['time','open','close','high','low','return',
         'ema10','ema20','ema50','ema200', 'ema100',
         'macd_slow', 'macd_fast',
         'rsi',
         'sto_main', 'sto_signal',
         'bb_mid', 'bb_down', 'bb_up',
         'cci','atr','willr', 'ADX_14', 'DMP_14', 'DMN_14'
         ]]
    i=i.dropna()
    df_res_full=pd.DataFrame()
    best_list_hyper=[]
    for a in range(len(is_oos_index)): # len(is_oos_index)
         daily_range=daily_range
         print('WIN:',a)
         train = i.iloc[i.index.isin(is_oos_index[a][0])]
         test = i.iloc[i.index.isin(is_oos_index[a][1])]
         train=train.reset_index(drop=True)
         test=test.reset_index(drop=True)
         dateindex=test[['time']]
         X_train=train[['ema10','ema20','ema50','ema200', 'ema100',
                       'macd_slow', 'macd_fast',
                       'rsi',
                       'sto_main', 'sto_signal',
                       'bb_mid', 'bb_down', 'bb_up',
                       'cci','atr','willr', 'ADX_14', 'DMP_14', 'DMN_14'
                        ]]
         y_train = train[['return']]
         X_test_close=test[['open','close']]
         X_test=test[['ema10','ema20','ema50','ema200', 'ema100',
                       'macd_slow', 'macd_fast',
                       'rsi',
                       'sto_main', 'sto_signal',
                       'bb_mid', 'bb_down', 'bb_up',
                       'cci','atr','willr', 'ADX_14', 'DMP_14', 'DMN_14'
                        ]]
         y_test = test[['return']]
         scaler = MinMaxScaler()
         X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
         X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
         train_indices=X_train.iloc[:daily_range].index.values
         valid_indices=X_train.iloc[daily_range:].index.values
         custom_cv = [[train_indices,valid_indices]]
         df_res_full_add=pd.DataFrame(dateindex,columns=['time'])
         df_res_full_add=df_res_full_add.join(X_test_close)
         df_res_full_add=df_res_full_add.join(y_test)
         scoring_list=['neg_mean_absolute_error','neg_mean_squared_error','neg_mean_absolute_percentage_error','r2'] # MAE, MSE, MAPE, R2
         model_grid = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, cv=custom_cv,return_train_score=True, scoring=scoring_list[0])
         model_grid.fit(X_train,y_train)
         predict_Model=model_grid.predict(X_test)
         quar75=y_train.quantile([0.75]).values # Threshold for signal
         quar25=y_train.quantile([0.25]).values # Threshold for signal
         predict_Model=pd.DataFrame(predict_Model, columns=['predict_Model'])
         df_res_full_add=df_res_full_add.join(predict_Model)
         df_res_full_add["quar75"]=float(quar75) # CHOOSE
         df_res_full_add["quar25"]=float(quar25) # CHOOSE
         df_res_full_add['mae_Model']=float(model_grid.cv_results_['mean_test_score'][0]*-1)
         df_res_full_add['mae_Model'] = mean_absolute_error(y_test, df_res_full_add['predict_Model'])
         best_list_hyper.append(model_grid.best_params_)
         best_list_hyper.append(m)
         df_res_full_add['signal_Model']=0

        # ------------- 1. Buy and Sell / Only buy /  Only sell
        #  df_res_full_add['signal_Model'][(df_res_full_add['predict_Model'] > 0)] = 1 # Only buy

        #  df_res_full_add['signal_Model'][(df_res_full_add['predict_Model'] < 0)] = -1 # Only sell

         df_res_full_add['signal_Model'][(df_res_full_add['predict_Model'] >= df_res_full_add['quar75'])] = 1 # Buy or Sell
         df_res_full_add['signal_Model'][(df_res_full_add['predict_Model'] <= df_res_full_add['quar25'])] = -1

         df_res_full=pd.concat([df_res_full,df_res_full_add])

    # ---------------------------------- Equity
    HYPERS.append(best_list_hyper)
    cost = 2 # 2 pip
    pipvalue=1 # based on lot size
    df_res_full['close_next']=df_res_full['close'].shift(-1)
    df_res_full['open_next']=df_res_full['open'].shift(-1)
    df_res_full['changes']=(df_res_full['close_next']-df_res_full['open_next'])*(10**2)*pipvalue
    df_res_full['equity_BAH'] = df_res_full['changes'].cumsum()+10000

    df_res_full['profit']=0
    df_res_full['cost']=0
    # df_res_full['cost'][df_res_full['signal_Model']!=0]=cost*pipvalue
    df_res_full['cost'][df_res_full['signal_Model']!=0]=df_res_full['close']*10000*0.0002
    df_res_full['cost'][df_res_full['signal_Model']==df_res_full['signal_Model'].shift(1)]=0
    df_res_full['changes_Model']=df_res_full['changes']-df_res_full['cost']

    df_res_full['profit'][(df_res_full['signal_Model'] == 1) &
                        (df_res_full['changes_Model'] > 0)] = abs(df_res_full['changes_Model'])
    df_res_full['profit'][(df_res_full['signal_Model'] == 1) &
                        (df_res_full['changes_Model'] < 0)] = abs(df_res_full['changes_Model'])*-1
    df_res_full['profit'][(df_res_full['signal_Model'] == -1) &
                        (df_res_full['changes_Model'] < 0)] = abs(df_res_full['changes_Model'])
    df_res_full['profit'][(df_res_full['signal_Model'] == -1) &
                        (df_res_full['changes_Model'] > 0)] = abs(df_res_full['changes_Model'])*-1
    df_res_full['profit'][df_res_full['signal_Model']==0]=0
    df_res_full['equity_Model'] = df_res_full['profit'].cumsum()+10000
    ####
    df_summary=pd.DataFrame([df_res_full['time'].iloc[0],
                             df_res_full['time'].iloc[-1],
                         10000, # Start
                         df_res_full['equity_Model'].iloc[-1],  # End
                         df_res_full['profit'][df_res_full['profit']>0].sum(), # Gross profit
                         df_res_full['profit'][df_res_full['profit']<0].sum(), # Gross loss
                         df_res_full['profit'][df_res_full['profit']>0].sum()+df_res_full['profit'][df_res_full['profit']<0].sum(), # Net profit
                         '{:.1%}'.format((df_res_full['profit'][df_res_full['profit']>0].sum()+df_res_full['profit'][df_res_full['profit']<0].sum())/10000), # Return on initial capital
                         df_res_full['profit'][df_res_full['profit']>0].sum()/df_res_full['profit'][df_res_full['profit']<0].sum(), # Profit factor
                         df_res_full['cost'][df_res_full['cost']!=0].count(), # Total number of trades
                         '{:.1%}'.format((df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['signal_Model']==-1)].count())/(df_res_full['cost'][df_res_full['cost']!=0].count())), # % sell
                         '{:.1%}'.format((df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['signal_Model']==1)].count())/(df_res_full['cost'][df_res_full['cost']!=0].count())), # % buy
                         df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']>0)].count(), # winning trades
                         df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']<0)].count(), # Lossing trades
                         '{:.1%}'.format((df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']>0)].count())/df_res_full['cost'][df_res_full['cost']!=0].count()), #%profitable
                         (df_res_full['profit'][df_res_full['profit']>0].sum()+df_res_full['profit'][df_res_full['profit']<0].sum())/(df_res_full['cost'][df_res_full['cost']!=0].count()), # Total number of trades
                         (df_res_full['profit'][df_res_full['profit']>0].sum())/(df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']>0)].count()), # Avg.winning trade
                         (df_res_full['profit'][df_res_full['profit']<0].sum())/(df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']<0)].count()), # Avg.lossing trade
                         ((df_res_full['profit'][df_res_full['profit']>0].sum())/(df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']>0)].count()))/((df_res_full['profit'][df_res_full['profit']<0].sum())/(df_res_full['cost'][(df_res_full['cost']!=0) & (df_res_full['profit']<0)].count())), # Ratio
                         df_res_full['profit'].max(), # Largest winning trade
                         df_res_full['profit'].min(), # Largest lossing trade
                         df_res_full['cost'].sum(),
                         calculate_max_drawdown((df_res_full['equity_Model']-df_res_full['equity_Model'].shift(1))/df_res_full['equity_Model'].shift(1)),
                         calculate_sharpe_ratio((df_res_full['equity_Model']-df_res_full['equity_Model'].shift(1))/df_res_full['equity_Model'].shift(1),0),
                         calculate_roi(df_res_full['equity_Model']),
                         df_res_full['mae_Model'].mean()
                         ],
                        columns=[str(m)],
                        index=['start_date','end_date','Start','End','Gross profit','Gross loss','Total Net profit', 'Return on initial capital',
                               'Profit factor','Total number of trades',
                               '%sell','%buy','Winning trades','Lossing trades','%profitable','Avg.Trade Net profit',
                               'Avg.Winning trade','Avg.Lossing trade','Ratio Avg.Win:Avg.Loss','Largest Winning trade','Largest Lossing trade','Cost',
                               'MD', 'IR/SR','ROI','MAE'])
    df_summary_OUTPUTS.append(df_summary)
    df_res_full=df_res_full.rename(columns = {'equity_Model':str(m)})
    OUTPUTS.append(df_res_full[['time','equity_BAH',str(m)]])

df_outputs=pd.concat([OUTPUTS[0]['time'].reset_index(),
    OUTPUTS[0]['equity_BAH'].reset_index(),
    OUTPUTS[0]['RIDGE'].reset_index(),
                     OUTPUTS[1]['SVM'].reset_index(),
                     OUTPUTS[2]['KNN'].reset_index(),
                     OUTPUTS[3]['RF'].reset_index(),
                     OUTPUTS[4]['XGBOOST'].reset_index()
                    #  OUTPUTS[5]['GBDT'].reset_index()
                     ],axis=1) # 'RIDGE', 'SVM','KNN','RF','XGBOOST','GBDT', 'ANN','LSTM','GRU'

df_outputs = df_outputs.T.drop_duplicates().T
df_summary_OUTPUTS = pd.concat(df_summary_OUTPUTS,axis=1)
print(df_summary_OUTPUTS)

# ---------------------------------- MAs
check=pd.DataFrame(columns=['short','long', 'SRs','iter'])
for a in range(len(is_oos_index)): # len(is_oos_index)
         train = i.iloc[i.index.isin(is_oos_index[a][0])]
         test = i.iloc[i.index.isin(is_oos_index[a][1])]
         train=train.reset_index(drop=True)
         test=test.reset_index(drop=True)
         both=pd.concat([train])
         SR_list=[]
         best_com=pd.DataFrame(columns=['short','long', 'SRs'])
         for short, long in zip(['ema10','ema10','ema10','ema10',
                        'ema20','ema20','ema20',
                        'ema50','ema50',
                        'ema100'],
                         ['ema20','ema50','ema100','ema200',
                          'ema50','ema100','ema200',
                          'ema100','ema200',
                          'ema200']):
             name_in=str(short+long)
             MA_cross = both[['time','open','close',short,long]]
             MA_cross['signal_Model']=0
                     # ------------- 2. Buy and Sell / Only buy /  Only sell
             MA_cross['signal_Model'][MA_cross[long]<MA_cross[short]]=1
             MA_cross['signal_Model'][MA_cross[long]>MA_cross[short]]=-1
             pipvalue=1 # based on lot size
             MA_cross['close_next']=MA_cross['close'].shift(-1)
             MA_cross['open_next']=MA_cross['open'].shift(-1)
             MA_cross['changes']=(MA_cross['close_next']-MA_cross['open_next'])*(10**2)*pipvalue
             MA_cross['profit']=0
             MA_cross['cost']=0
             # MA_cross['cost'][MA_cross['signal_Model']!=0]=cost*pipvalue
             MA_cross['cost'][MA_cross['signal_Model']!=0]=MA_cross['close']*10000*0.0002
             MA_cross['cost'][MA_cross['signal_Model']==MA_cross['signal_Model'].shift(1)]=0
             MA_cross['changes_Model']=MA_cross['changes']-MA_cross['cost']

             MA_cross['profit'][(MA_cross['signal_Model'] == 1) &
                                 (MA_cross['changes_Model'] > 0)] = abs(MA_cross['changes_Model'])
             MA_cross['profit'][(MA_cross['signal_Model'] == 1) &
                                 (MA_cross['changes_Model'] < 0)] = abs(MA_cross['changes_Model'])*-1
             MA_cross['profit'][(MA_cross['signal_Model'] == -1) &
                                 (MA_cross['changes_Model'] < 0)] = abs(MA_cross['changes_Model'])
             MA_cross['profit'][(MA_cross['signal_Model'] == -1) &
                                 (MA_cross['changes_Model'] > 0)] = abs(MA_cross['changes_Model'])*-1
             MA_cross['profit'][MA_cross['signal_Model']==0]=0
             MA_cross['equity_Model'] = MA_cross['profit'].cumsum()+10000
             SR=calculate_sharpe_ratio((MA_cross['equity_Model']-MA_cross['equity_Model'].shift(1))/MA_cross['equity_Model'].shift(1),0)
             SR_list.append(SR)
             best_com=best_com._append({'short': short,'long': long, 'SRs': SR,'iter':a}, ignore_index=True)
         x=best_com[best_com.SRs==max(best_com.SRs)]
         check=check._append(x)

MA_res=pd.DataFrame(columns=['time', 'open', 'close', 'short', 'long', 'chosen','signal_Model'])
for a in range(len(is_oos_index)): # len(is_oos_index)
         i=i[i.time>='2003-01-01']
         i=i[i.time<='2023-06-30']
         train = i.iloc[i.index.isin(is_oos_index[a][0])]
         test = i.iloc[i.index.isin(is_oos_index[a][1])]
         train=train.reset_index(drop=True)
         test=test.reset_index(drop=True)
         MA_cross = test[['time','open','close',check.iloc[a]['short'],check.iloc[a]['long']]]
        #  print(MA_cross.columns)
         MA_cross.columns.values[-2] = "short"
         MA_cross.columns.values[-1] = "long"
         MA_cross['signal_Model']=0
                 # ------------- 3. Buy and Sell / Only buy /  Only sell
         MA_cross['signal_Model'][MA_cross['long']<MA_cross["short"]]=1
         MA_cross['signal_Model'][MA_cross["long"]>MA_cross["short"]]=-1
         MA_cross['chosen']=check.iloc[a]['short']+check.iloc[a]['long']
         MA_res=pd.concat([MA_res,MA_cross])

pipvalue=1 # based on lot size
MA_res['close_next']=MA_res['close'].shift(-1)
MA_res['open_next']=MA_res['open'].shift(-1)
MA_res['changes']=(MA_res['close_next']-MA_res['open_next'])*(10**2)*pipvalue
MA_res['profit']=0
MA_res['cost']=0
# MA_res['cost'][MA_res['signal_Model']!=0]=cost*pipvalue
MA_res['cost'][MA_res['signal_Model']!=0]=MA_res['close']*10000*0.0002
MA_res['cost'][MA_res['signal_Model']==MA_res['signal_Model'].shift(1)]=0
MA_res['changes_Model']=MA_res['changes']-MA_res['cost']
MA_res['profit'][(MA_res['signal_Model'] == 1) &
                    (MA_res['changes_Model'] > 0)] = abs(MA_res['changes_Model'])
MA_res['profit'][(MA_res['signal_Model'] == 1) &
                    (MA_res['changes_Model'] < 0)] = abs(MA_res['changes_Model'])*-1
MA_res['profit'][(MA_res['signal_Model'] == -1) &
                    (MA_res['changes_Model'] < 0)] = abs(MA_res['changes_Model'])
MA_res['profit'][(MA_res['signal_Model'] == -1) &
                    (MA_res['changes_Model'] > 0)] = abs(MA_res['changes_Model'])*-1
MA_res['profit'][MA_res['signal_Model']==0]=0
MA_res['equity_Model'] = MA_res['profit'].cumsum()+10000
df_summary_MAs=pd.DataFrame([MA_res['time'].iloc[0],
                        MA_res['time'].iloc[-1],
                    10000, # Start
                    MA_res['equity_Model'].iloc[-1],  # End
                    MA_res['profit'][MA_res['profit']>0].sum(), # Gross profit
                    MA_res['profit'][MA_res['profit']<0].sum(), # Gross loss
                    MA_res['profit'][MA_res['profit']>0].sum()+MA_res['profit'][MA_res['profit']<0].sum(), # Net profit
                    '{:.1%}'.format((MA_res['profit'][MA_res['profit']>0].sum()+MA_res['profit'][MA_res['profit']<0].sum())/10000), # Return on initial capital
                    MA_res['profit'][MA_res['profit']>0].sum()/MA_res['profit'][MA_res['profit']<0].sum(), # Profit factor
                    MA_res['cost'][MA_res['cost']!=0].count(), # Total number of trades
                    '{:.1%}'.format((MA_res['cost'][(MA_res['cost']!=0) & (MA_res['signal_Model']==-1)].count())/(MA_res['cost'][MA_res['cost']!=0].count())), # % sell
                    '{:.1%}'.format((MA_res['cost'][(MA_res['cost']!=0) & (MA_res['signal_Model']==1)].count())/(MA_res['cost'][MA_res['cost']!=0].count())), # % buy
                    MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']>0)].count(), # winning trades
                    MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']<0)].count(), # Lossing trades
                    '{:.1%}'.format((MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']>0)].count())/MA_res['cost'][MA_res['cost']!=0].count()), #%profitable
                    (MA_res['profit'][MA_res['profit']>0].sum()+MA_res['profit'][MA_res['profit']<0].sum())/(MA_res['cost'][MA_res['cost']!=0].count()), # Total number of trades
                    (MA_res['profit'][MA_res['profit']>0].sum())/(MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']>0)].count()), # Avg.winning trade
                    (MA_res['profit'][MA_res['profit']<0].sum())/(MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']<0)].count()), # Avg.lossing trade
                    ((MA_res['profit'][MA_res['profit']>0].sum())/(MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']>0)].count()))/((MA_res['profit'][MA_res['profit']<0].sum())/(MA_res['cost'][(MA_res['cost']!=0) & (MA_res['profit']<0)].count())), # Ratio
                    MA_res['profit'].max(), # Largest winning trade
                    MA_res['profit'].min(), # Largest lossing trade
                    MA_res['cost'].sum(),
                    calculate_max_drawdown((MA_res['equity_Model']-MA_res['equity_Model'].shift(1))/MA_res['equity_Model'].shift(1)),
                    calculate_sharpe_ratio((MA_res['equity_Model']-MA_res['equity_Model'].shift(1))/MA_res['equity_Model'].shift(1),0),
                    calculate_roi(MA_res['equity_Model'])
                    ],
                    columns=[str('MAs')],
                    index=['start_date','end_date','Start','End','Gross profit','Gross loss','Total Net profit', 'Return on initial capital',
                          'Profit factor','Total number of trades',
                          '%sell','%buy','Winning trades','Lossing trades','%profitable','Avg.Trade Net profit',
                          'Avg.Winning trade','Avg.Lossing trade','Ratio Avg.Win:Avg.Loss','Largest Winning trade','Largest Lossing trade','Cost',
                          'MD', 'IR/SR','ROI'])
df_summary_MAs
# ---------------------------------- Benchmark
i=i[i.time>=MA_res['time'].iloc[0]]
i=i[i.time<=MA_res['time'].iloc[-1]]
benchmark = i[['time','open','close']]
benchmark['signal_Model']=0
benchmark['signal_Model']=1
benchmark['signal_Model'].iloc[-1]=-1

pipvalue=1 # based on lot size
benchmark['close_next']=benchmark['close'].shift(-1)
benchmark['open_next']=benchmark['open'].shift(-1)
benchmark['changes']=(benchmark['close_next']-benchmark['open_next'])*(10**2)*pipvalue
benchmark['profit']=0
benchmark['cost']=0
# benchmark['cost'][benchmark['signal_Model']!=0]=cost*pipvalue
benchmark['cost'][benchmark['signal_Model']!=0]=benchmark['close']*10000*0.0002
benchmark['cost'][benchmark['signal_Model']==benchmark['signal_Model'].shift(1)]=0
benchmark['changes_Model']=benchmark['changes']-benchmark['cost']
benchmark['profit'][(benchmark['signal_Model'] == 1) &
                    (benchmark['changes_Model'] > 0)] = abs(benchmark['changes_Model'])
benchmark['profit'][(benchmark['signal_Model'] == 1) &
                    (benchmark['changes_Model'] < 0)] = abs(benchmark['changes_Model'])*-1
benchmark['profit'][(benchmark['signal_Model'] == -1) &
                    (benchmark['changes_Model'] < 0)] = abs(benchmark['changes_Model'])
benchmark['profit'][(benchmark['signal_Model'] == -1) &
                    (benchmark['changes_Model'] > 0)] = abs(benchmark['changes_Model'])*-1
benchmark['profit'][benchmark['signal_Model']==0]=0
benchmark['equity_Model'] = benchmark['profit'].cumsum()+10000
df_summary_benchmark=pd.DataFrame([benchmark['time'].iloc[0],
                        benchmark['time'].iloc[-1],
                    10000, # Start
                    benchmark['equity_Model'].iloc[-1],  # End
                    benchmark['profit'][benchmark['profit']>0].sum(), # Gross profit
                    benchmark['profit'][benchmark['profit']<0].sum(), # Gross loss
                    benchmark['profit'][benchmark['profit']>0].sum()+benchmark['profit'][benchmark['profit']<0].sum(), # Net profit
                    '{:.1%}'.format((benchmark['profit'][benchmark['profit']>0].sum()+benchmark['profit'][benchmark['profit']<0].sum())/10000), # Return on initial capital
                    benchmark['profit'][benchmark['profit']>0].sum()/benchmark['profit'][benchmark['profit']<0].sum(), # Profit factor
                    benchmark['cost'][benchmark['cost']!=0].count(), # Total number of trades
                    '{:.1%}'.format((benchmark['cost'][(benchmark['cost']!=0) & (benchmark['signal_Model']==-1)].count())/(benchmark['cost'][benchmark['cost']!=0].count())), # % sell
                    '{:.1%}'.format((benchmark['cost'][(benchmark['cost']!=0) & (benchmark['signal_Model']==1)].count())/(benchmark['cost'][benchmark['cost']!=0].count())), # % buy
                    benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']>0)].count(), # winning trades
                    benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']<0)].count(), # Lossing trades
                    '{:.1%}'.format((benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']>0)].count())/benchmark['cost'][benchmark['cost']!=0].count()), #%profitable
                    (benchmark['profit'][benchmark['profit']>0].sum()+benchmark['profit'][benchmark['profit']<0].sum())/(benchmark['cost'][benchmark['cost']!=0].count()), # Total number of trades
                    (benchmark['profit'][benchmark['profit']>0].sum())/(benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']>0)].count()), # Avg.winning trade
                    (benchmark['profit'][benchmark['profit']<0].sum())/(benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']<0)].count()), # Avg.lossing trade
                    ((benchmark['profit'][benchmark['profit']>0].sum())/(benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']>0)].count()))/((benchmark['profit'][benchmark['profit']<0].sum())/(benchmark['cost'][(benchmark['cost']!=0) & (benchmark['profit']<0)].count())), # Ratio
                    benchmark['profit'].max(), # Largest winning trade
                    benchmark['profit'].min(), # Largest lossing trade
                    benchmark['cost'].sum(),
                    calculate_max_drawdown((benchmark['equity_Model']-benchmark['equity_Model'].shift(1))/benchmark['equity_Model'].shift(1)),
                    calculate_sharpe_ratio((benchmark['equity_Model']-benchmark['equity_Model'].shift(1))/benchmark['equity_Model'].shift(1),0),
                    calculate_roi(benchmark['equity_Model'])
                    ],
                    columns=['benchmark'],
                    index=['start_date','end_date','Start','End','Gross profit','Gross loss','Total Net profit', 'Return on initial capital',
                          'Profit factor','Total number of trades',
                          '%sell','%buy','Winning trades','Lossing trades','%profitable','Avg.Trade Net profit',
                          'Avg.Winning trade','Avg.Lossing trade','Ratio Avg.Win:Avg.Loss','Largest Winning trade','Largest Lossing trade','Cost',
                          'MD', 'IR/SR','ROI'])
df_summary_benchmark
#####################################################
table=pd.concat([df_summary_OUTPUTS,df_summary_MAs,df_summary_benchmark],axis=1)
MA_res.columns = [*MA_res.columns[:-1], 'equity_MA']
benchmark.columns = [*benchmark.columns[:-1], 'equity_benchmark']
equity=pd.concat([df_outputs,MA_res[['time','equity_MA']].reset_index(),benchmark[['time','equity_benchmark']].reset_index()],axis=1)
hyper=HYPERS

train+validation:  0 781
test: 782 1042
WIN: 0
WIN: 1
WIN: 2
WIN: 3
WIN: 4
WIN: 5
WIN: 6
WIN: 7
WIN: 8
WIN: 9
WIN: 10
WIN: 11
WIN: 12
WIN: 13
WIN: 14
WIN: 15
WIN: 16
WIN: 17
WIN: 18
WIN: 19
WIN: 20
WIN: 21
WIN: 22
WIN: 23
WIN: 24
WIN: 25
WIN: 26
WIN: 27
WIN: 28
WIN: 29
WIN: 30
WIN: 31
WIN: 32
WIN: 33
WIN: 34
WIN: 35
WIN: 36
WIN: 37
WIN: 38
WIN: 0
WIN: 1
WIN: 2
WIN: 3
WIN: 4
WIN: 5
WIN: 6
WIN: 7
WIN: 8
WIN: 9
WIN: 10
WIN: 11
WIN: 12
WIN: 13
WIN: 14
WIN: 15
WIN: 16
WIN: 17
WIN: 18
WIN: 19
WIN: 20
WIN: 21
WIN: 22
WIN: 23
WIN: 24
WIN: 25
WIN: 26
WIN: 27
WIN: 28
WIN: 29
WIN: 30
WIN: 31
WIN: 32
WIN: 33
WIN: 34
WIN: 35
WIN: 36
WIN: 37
WIN: 38
WIN: 0
WIN: 1
WIN: 2
WIN: 3
WIN: 4
WIN: 5
WIN: 6
WIN: 7
WIN: 8
WIN: 9
WIN: 10
WIN: 11
WIN: 12
WIN: 13
WIN: 14
WIN: 15
WIN: 16
WIN: 17
WIN: 18
WIN: 19
WIN: 20
WIN: 21
WIN: 22
WIN: 23
WIN: 24
WIN: 25
WIN: 26
WIN: 27
WIN: 28
WIN: 29
WIN: 30
WIN: 31
WIN: 32
WIN: 33
WIN: 34
WIN: 35
WIN: 36
WIN: 37
WIN: 38
WIN: 0
WIN: 1
WIN: 2
WIN: 3
WIN: 4
WIN: 5
WIN: 6
WIN: 